In [2]:
# импорт необходимых библиотек
import pickle
import pandas as pd
from flask import Flask, request

In [3]:
# загружаем необходимые модули
with open('encoder.pcl', 'rb') as f:
    encoder = pickle.load(f)
    
with open('model.pcl', 'rb') as f:
    model = pickle.load(f)

# Разработка API

Определю команды(функции), которые будут доступны в API

In [4]:
app = Flask(__name__)

@app.route('/')
def check_server():
    return '''
    Сервер успешно запущен<br>
    Доступные команды:<br>
        /model/prediction/(file_name)/ - (file_name) имя файля по которому нужно сделать предсказание, формат файла должен быть .csv<br>
    '''

@app.route('/model/prediction/<string:file_name>/')
def get_predict(file_name):
    X_df = pd.read_csv('data/'+ file_name)
    
    X_df['age'] = encoder['age'].transform(X_df['age'])
    X_df['A1Cresult'] = encoder['A1Cresult'].transform(X_df['A1Cresult'])
    X_df['insulin'] = encoder['insulin'].transform(X_df['insulin'])
    
    pred = model.predict(X_df)
    res = encoder['readmitted'].inverse_transform(pred)[0] if encoder['readmitted'].inverse_transform(pred)[0] != 'NO' else '0'

    return f'Наиболее вероятно, что пациент будет повторно принят: {res} раз'

### Запуск сервера

Чтобы перейти на гл. страницу перейдите по ссылке ниже.

In [ ]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [30/Nov/2020 11:08:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Nov/2020 11:08:29] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [30/Nov/2020 11:08:40] "GET /model/prediction/example.csv/ HTTP/1.1" 200 -


Для проверки роботоспособности API можно перейти по ссылке: http://127.0.0.1:5000/model/prediction/example.csv/